## Step by step buidling of a knowledge graph

The goal of this notebook is to build an example of knowledge graph using a step by step approach.


## Prerequisites

This notebook assumes you've created a project within the sandbox deployment of Nexus. If not follow the Blue Brain Nexus [Quick Start tutorial](https://bluebrain.github.io/nexus-bbp-domains/docs/bbptutorial/getting-started/quick-start/index.html).

## Overview
Explain the Research domain (with figure). Uses schema.org and json-ld to describe entities

You'll work through the following steps:

1. Create and configure a Blue Brain Nexus client
2. Create a Person entity
3. Create an Organization entity and link it to the Person entity as affiliation
4. Create an article entity and link it to the Person entity as author
5. Explore and navigate the created knowledge graph

## Step 1: Create and configure a Nexus client

In [2]:
#install the Blue Brain Nexus python SDK
!pip install -U nexus-sdk

Requirement already up-to-date: nexus-sdk in /Users/mfsy/anaconda3/lib/python3.6/site-packages (0.2.1)
Requirement not upgraded as not directly required: requests in /Users/mfsy/anaconda3/lib/python3.6/site-packages (from nexus-sdk) (2.21.0)
Requirement not upgraded as not directly required: certifi>=2017.4.17 in /Users/mfsy/anaconda3/lib/python3.6/site-packages (from requests->nexus-sdk) (2019.3.9)
Requirement not upgraded as not directly required: idna<2.9,>=2.5 in /Users/mfsy/anaconda3/lib/python3.6/site-packages (from requests->nexus-sdk) (2.8)
Requirement not upgraded as not directly required: urllib3<1.25,>=1.21.1 in /Users/mfsy/anaconda3/lib/python3.6/site-packages (from requests->nexus-sdk) (1.24.3)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /Users/mfsy/anaconda3/lib/python3.6/site-packages (from requests->nexus-sdk) (3.0.4)
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.1.1 is a

In [3]:
#Set a token to authenticate to Nexus
import getpass
token = getpass.getpass()

········


### Create a nexus client. Don't forget to replace \$PROJECTLABEL with your own project label (project=...)

In [20]:
#Configure a nexus client
nexus_environment = "https://sandbox.bluebrainnexus.io/v1"
org ="demo"
project ="$PROJECTLABEL"

import nexussdk as nexus
nexus.config.set_environment(nexus_environment)
nexus.config.set_token(token)

vocab = "%s/vocabs/%s/%s/"%(nexus_environment, org, project)



In [5]:
from pygments.lexers import JsonLdLexer
from pygments import highlight
from pygments.formatters import TerminalFormatter, TerminalTrueColorFormatter
import json

In [6]:
from sparqlendpointhelper import SparqlViewHelper

%load_ext autoreload
%autoreload 1
%aimport sparqlendpointhelper
%aimport utils
%reload_ext autoreload


### Create a SPARQL helper to be used for running queries 

In [7]:
sparqlview_endpoint = nexus_environment+"/views/"+org+"/"+project+"/graph/sparql"
sparqlviewhelper = SparqlViewHelper(sparqlview_endpoint,nexus_environment, org, project, token)

### Import the Blue Brain Knowledge Graph schema 

The following calls will create a jsonld context (identified by https://bbp.neuroshapes.org) resource importing the Blue Brain Knowledge Graph schema (identified by https://neuroshapes.org and available [here](https://sandbox.bluebrainnexus.io/web/neurosciencegraph/datamodels/resources/https%3A%2F%2Fneuroshapes.org)). The schema here is made of a vocabulary.

In [10]:
#Import the BBP Knowledge Graph schema

project_to_resolve_to = "neurosciencegraph/datamodels"

# Resolver
cross_project_resolver = {
  "@type": [
    "CrossProject"
  ],
  "projects": [
    project_to_resolve_to
  ],
  "identities": [
    {
    "@type": "Authenticated",
    "realm": "github"
    }
  ],
  "priority": 50
}


response = utils.create_resolver(nexus,cross_project_resolver,org,project)
utils.pretty_print(response)


{
    "@context": "https://bluebrain.github.io/nexus/contexts/resource.json",
    "@id": "https://sandbox.bluebrainnexus.io/v1/resources/demo/awesomeproject/_/9f20a964-dfdc-4f23-a3d9-aa69e12a982d",
    "@type": [
        "Resolver",
        "CrossProject"
    ],
    "_self": "https://sandbox.bluebrainnexus.io/v1/resolvers/demo/awesomeproject/9f20a964-dfdc-4f23-a3d9-aa69e12a982d",
    "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/resolver.json",
    "_project": "https://sandbox.bluebrainnexus.io/v1/projects/demo/awesomeproject",
    "_rev": 1,
    "_deprecated": false,
    "_createdAt": "2019-06-07T11:54:58.799Z",
    "_createdBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy",
    "_updatedAt": "2019-06-07T11:54:58.799Z",
    "_updatedBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy"
}



In [ ]:
bbp_context =  {
    "@context":[
        "https://neuroshapes.org",
        {
            "@vocab": vocab
        }
    ],
    "@id":"https://bbp.neuroshapes.org"
    
}

response = utils.create_resource(nexus,bbp_context,org,project, )
utils.pretty_print(response)


## Step 2: Create a Person entity

Let define an entity of type Person as follows:

* A person is an entity of type Person (@type value)
* A person has an identifier (@id value)
* A person has a family name (familyName value), a given name (givenName value) and a job title (jobTitle value)
* A person has a job


In [9]:
# Use an orcid identifier if you have one, or your github id
person ={
    "@context":"https://bbp.neuroshapes.org",
    "@id":"http://your/id/here",
    "@type":"Person",
    "familyName":"your familly name here",
    "givenName":"your given name here",
    "jobTitle":"job title"
} 

response = utils.create_resource(nexus,person,org,project)
utils.pretty_print(response)


{
    "@context": "https://bluebrain.github.io/nexus/contexts/resource.json",
    "@id": "http://your/id/here",
    "@type": "http://schema.org/Person",
    "_self": "https://sandbox.bluebrainnexus.io/v1/resources/demo/awesomeproject/_/http%3A%2F%2Fyour%2Fid%2Fhere",
    "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/unconstrained.json",
    "_project": "https://sandbox.bluebrainnexus.io/v1/projects/demo/awesomeproject",
    "_rev": 1,
    "_deprecated": false,
    "_createdAt": "2019-06-07T11:54:33.112Z",
    "_createdBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy",
    "_updatedAt": "2019-06-07T11:54:33.112Z",
    "_updatedBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy"
}



## Step 3: Create an article entity and link it to the Person entity as author

The knowledge Graph now contains a single entity of type Person. Let add to the knowledge graph one scholarly article (publication) authored by the person entity:

* A scholarly article is  an entity of type ScholarlyArticle (@type value)
* A scholarly article has an identifier (@id value)
* A scholarly article has a name (name value)
* A scholarly article has a publishing date (datePublished value)


###  Create a ScholarlyArticle entity

In [12]:
#Let create an entity describing a publication with a doi: https://doi.org/10.1186/1471-2105-13-s1-s4
scholarly_article ={
    "@context":"https://bbp.neuroshapes.org",
    "@type":"ScholarlyArticle",
    "@id":"https://doi.org/10.1016/j.cell.2015.09.029",
    "name":"Reconstruction and Simulation of Neocortical Microcircuitry",
    "datePublished":"2015-10"
} 

response = utils.create_resource(nexus,scholarly_article,org,project)
utils.pretty_print(response)

{
    "@context": "https://bluebrain.github.io/nexus/contexts/resource.json",
    "@id": "https://doi.org/10.1016/j.cell.2015.09.029",
    "@type": "https://sandbox.bluebrainnexus.io/v1/vocabs/demo/awesomeproject/ScholarlyArticle",
    "_self": "https://sandbox.bluebrainnexus.io/v1/resources/demo/awesomeproject/_/https%3A%2F%2Fdoi.org%2F10.1016%2Fj.cell.2015.09.029",
    "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/unconstrained.json",
    "_project": "https://sandbox.bluebrainnexus.io/v1/projects/demo/awesomeproject",
    "_rev": 1,
    "_deprecated": false,
    "_createdAt": "2019-06-07T11:55:29.238Z",
    "_createdBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy",
    "_updatedAt": "2019-06-07T11:55:29.238Z",
    "_updatedBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy"
}



###  Link the Person and the ScholarlyArticle entity with authorship

In [13]:
# A reference to the Person identifier (value of @id) is enough to link with the article
# Note the revision value change (should be "_rev": 2) after an update
scholarly_article["author"]=person["@id"]
response = utils.update_resource(nexus,scholarly_article["@id"],scholarly_article,org,project)
utils.pretty_print(response)

{
    "@context": "https://bluebrain.github.io/nexus/contexts/resource.json",
    "@id": "https://doi.org/10.1016/j.cell.2015.09.029",
    "@type": "https://sandbox.bluebrainnexus.io/v1/vocabs/demo/awesomeproject/ScholarlyArticle",
    "_self": "https://sandbox.bluebrainnexus.io/v1/resources/demo/awesomeproject/_/https%3A%2F%2Fdoi.org%2F10.1016%2Fj.cell.2015.09.029",
    "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/unconstrained.json",
    "_project": "https://sandbox.bluebrainnexus.io/v1/projects/demo/awesomeproject",
    "_rev": 2,
    "_deprecated": false,
    "_createdAt": "2019-06-07T11:55:29.238Z",
    "_createdBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy",
    "_updatedAt": "2019-06-07T11:55:34.337Z",
    "_updatedBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy"
}



###  Fetch the ScholarlyArticle by identifier to view the update 

In [14]:
response = utils.fetch_resource(nexus,scholarly_article["@id"],org,project)
utils.pretty_print(response)

{
    "@context": [
        "https://bbp.neuroshapes.org",
        "https://bluebrain.github.io/nexus/contexts/resource.json"
    ],
    "@id": "https://doi.org/10.1016/j.cell.2015.09.029",
    "@type": "ScholarlyArticle",
    "author": "http://your/id/here",
    "datePublished": "2015-10",
    "name": "Reconstruction and Simulation of Neocortical Microcircuitry",
    "_self": "https://sandbox.bluebrainnexus.io/v1/resources/demo/awesomeproject/_/https%3A%2F%2Fdoi.org%2F10.1016%2Fj.cell.2015.09.029",
    "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/unconstrained.json",
    "_project": "https://sandbox.bluebrainnexus.io/v1/projects/demo/awesomeproject",
    "_rev": 2,
    "_deprecated": false,
    "_createdAt": "2019-06-07T11:55:29.238Z",
    "_createdBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy",
    "_updatedAt": "2019-06-07T11:55:34.337Z",
    "_updatedBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy"
}



##  Step 4: Update the Person entity to add its affiliation

Let update the person entity with an affiliation information. We will use EPFL as an affiliation and link to the person entity via "affiliation" propoerty.

In this step, we'll:

* Search for the EPFL organization entity and retrieve its identifier
* Update the person entity affiliation to point to the EPFL entity

### Search for the EPFL organization entity and retrieve its identifier and name

The query below is a SPARQL query. Most SPARQL queries you'll see will have the anotomy above with:

* a SELECT clause that let you select the variables you want to retrieve
* a WHERE clause defining a set of constraints that the variables should satisfy to be retrieved
* LIMIT and OFFSET clauses to enable pagination
* the constraints are usually graph patterns in the form of triple (?s for subject, ?p for property and ?o for ?object)

In [15]:
acronym = "\"EPFL\""


epfl_query = """
Select  ?institute  ?name ?grid_id
WHERE {
    ?institute vocab:acronym %s.
    ?institute a vocab:Organization.
    ?institute vocab:grid_id ?grid_id . 
    ?institute vocab:name ?name . 
}
LIMIT 100
""" % (acronym)

orgs_df = sparqlviewhelper.query_sparql(epfl_query,result_format = "DATAFRAME")
display(orgs_df.head(100))

,institute,name,grid_id
0,http://www.grid.ac/institutes/grid.5333.6,École Polytechnique Fédérale de Lausanne,grid.5333.6


### Update the person entity affiliation to point to the EPFL entity

In [16]:
# Note the revision value change after an update
person["affiliation"]=orgs_df["institute"]

for index, row in orgs_df.iterrows():
    person["affiliation"]=row["institute"]


response = utils.update_resource(nexus,person["@id"],person,org,project)
utils.pretty_print(response)

{
    "@context": "https://bluebrain.github.io/nexus/contexts/resource.json",
    "@id": "http://your/id/here",
    "@type": "http://schema.org/Person",
    "_self": "https://sandbox.bluebrainnexus.io/v1/resources/demo/awesomeproject/_/http%3A%2F%2Fyour%2Fid%2Fhere",
    "_constrainedBy": "https://bluebrain.github.io/nexus/schemas/unconstrained.json",
    "_project": "https://sandbox.bluebrainnexus.io/v1/projects/demo/awesomeproject",
    "_rev": 2,
    "_deprecated": false,
    "_createdAt": "2019-06-07T11:54:33.112Z",
    "_createdBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy",
    "_updatedAt": "2019-06-07T11:55:45.265Z",
    "_updatedBy": "https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy"
}



## Setp 5 Explore and navigate the created knowledge graph using the SPARQL query language

Let write our first query.

In [17]:
select_all_query = """
SELECT ?s ?p ?o
WHERE
{
  ?s ?p ?o
}
OFFSET 0
LIMIT 5
"""

results_df = sparqlviewhelper.query_sparql(select_all_query,result_format = "DATAFRAME")

results_df.head()

,s,p,o
0,https://bluebrain.github.io/nexus/vocabulary/defaultElasticSearchIndex,https://bluebrain.github.io/nexus/vocabulary/constrainedBy,https://bluebrain.github.io/nexus/schemas/view.json
1,https://bluebrain.github.io/nexus/vocabulary/defaultElasticSearchIndex,https://bluebrain.github.io/nexus/vocabulary/createdAt,2019-06-05T08:50:05.838Z
2,https://bluebrain.github.io/nexus/vocabulary/defaultElasticSearchIndex,https://bluebrain.github.io/nexus/vocabulary/createdBy,https://sandbox.bluebrainnexus.io/v1/realms/github/users/annakristinkaufmann
3,https://bluebrain.github.io/nexus/vocabulary/defaultElasticSearchIndex,https://bluebrain.github.io/nexus/vocabulary/deprecated,false
4,https://bluebrain.github.io/nexus/vocabulary/defaultElasticSearchIndex,https://bluebrain.github.io/nexus/vocabulary/includeMetadata,true


Most SPARQL queries you'll see will have the anotomy above with:
* a **SELECT** clause that let you select the variables you want to retrieve
* a **WHERE** clause defining a set of constraints that the variables should satisfy to be retrieved
* **LIMIT** and **OFFSET** clauses to enable pagination
* the constraints are usually graph patterns in the form of **triple** (?s for subject, ?p for property and ?o for ?object)

Multiple triples can be provided as graph pattern to match but each triple should end with a period. As an example, let retrieve EPFL (?institute) along with its name (?name) and identifier (?grid_id).

In [18]:
epfl_with_name = """

Select  ?institute ?name ?grid_id
WHERE {
    ?institute vocab:acronym "EPFL".
    ?institute a vocab:Organization.
    ?institute vocab:grid_id ?grid_id . 
    ?institute vocab:name ?name . 
}
LIMIT 100
"""

results_df = sparqlviewhelper.query_sparql(epfl_with_name,result_format = "DATAFRAME")
results_df.head()


,institute,name,grid_id
0,http://www.grid.ac/institutes/grid.5333.6,École Polytechnique Fédérale de Lausanne,grid.5333.6


This is a typical instance query where entities are filtered by their type(s) and then some of their properties are retrieved (here ?name). 

Let retrieve everything that is linked (outgoing) to the organizations. 
The * character in the SELECT clause indicates to retreve all variables: ?institute, ?p, ?o

In [19]:
org_with_properties = """
Select *
 WHERE  {
    ?institute a vocab:Organization.
    ?institute ?p ?o.
} LIMIT 20
"""

results_df = sparqlviewhelper.query_sparql(org_with_properties,result_format = "DATAFRAME")
results_df.head()


,institute,p,o
0,http://www.grid.ac/institutes/grid.1006.7,https://bluebrain.github.io/nexus/vocabulary/constrainedBy,https://bluebrain.github.io/nexus/schemas/unconstrained.json
1,http://www.grid.ac/institutes/grid.1006.7,https://bluebrain.github.io/nexus/vocabulary/createdAt,2019-06-05T08:55:35.170Z
2,http://www.grid.ac/institutes/grid.1006.7,https://bluebrain.github.io/nexus/vocabulary/createdBy,https://sandbox.bluebrainnexus.io/v1/realms/github/users/annakristinkaufmann
3,http://www.grid.ac/institutes/grid.1006.7,https://bluebrain.github.io/nexus/vocabulary/deprecated,false
4,http://www.grid.ac/institutes/grid.1006.7,https://bluebrain.github.io/nexus/vocabulary/project,https://sandbox.bluebrainnexus.io/v1/projects/demo/awesomeproject


As a little exercise, write a query retrieving incoming entities to organizations. You can copy past the query above and modify it.

Hints: ?s ?p ?o can be read as: ?o is linked to ?s with an outgoing link.

Do you have results ?

In [ ]:
#Your query here
